# AI Takeoff MVP - Inference/Testing Notebook

This notebook loads your trained YOLOv8 model and runs detection on test images to count construction elements.

## Prerequisites
- Trained model at `../models/best.pt`
- Test blueprint images to analyze

## 1. Import Libraries

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from PIL import Image

print("✅ Libraries imported successfully")

## 2. Load Trained Model

In [ ]:
# Path to your trained model
model_path = Path('../models/best.pt')

if not model_path.exists():
    print("❌ Model not found! Please run train_mvp.ipynb first.")
    print(f"Looking for model at: {model_path.absolute()}")
else:
    print(f"✅ Loading model from: {model_path.absolute()}")
    model = YOLO(str(model_path))
    print("✅ Model loaded successfully!")
    
    # Display model info
    print(f"\n📊 Model Information:")
    print(f"Classes: {model.names}")

## 3. Select Test Image

Choose an image to test. You can:
- Use an image from your training set (to verify the model works)
- Use a new, unseen blueprint image (to test real-world performance)

In [ ]:
# Option 1: Specify a direct path to your test image
test_image_path = '../data_raw/your_test_image.png'  # ⬅️ CHANGE THIS

# Option 2: Or select from labeled images
# Uncomment the lines below to use an image from your training set
# images_dir = Path('../data_labeled/images')
# image_files = list(images_dir.glob('*.png')) + list(images_dir.glob('*.jpg'))
# if image_files:
#     test_image_path = str(image_files[0])  # Use first image
#     print(f"Using image: {Path(test_image_path).name}")

print(f"Test image: {test_image_path}")

# Verify the image exists
if not Path(test_image_path).exists():
    print("\n⚠️  Image not found! Please update the test_image_path variable above.")
    print("\nAvailable images in data_labeled/images:")
    images_dir = Path('../data_labeled/images')
    if images_dir.exists():
        for img in images_dir.glob('*'):
            if img.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                print(f"  - {img.name}")

## 4. Run Detection

This is the core inference step - the model will detect objects and count them.

In [ ]:
# Run inference
print("🔍 Running detection...\n")

# Perform detection
# conf: confidence threshold (0.25 = 25% confidence minimum)
# iou: intersection over union threshold for NMS
results = model(test_image_path, conf=0.25, iou=0.45)

print("✅ Detection complete!")

## 5. Extract and Display Results

### 🎯 This is the MVP deliverable: Object count for takeoff!

In [ ]:
# Process results
for result in results:
    # Get bounding boxes
    boxes = result.boxes
    
    # Total count
    total_count = len(boxes)
    
    # Count by class
    class_counts = {}
    for box in boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
    
    # Display results
    print("="*50)
    print("🎯 TAKEOFF RESULTS")
    print("="*50)
    print(f"\n📊 Total Objects Detected: {total_count}")
    print("\n📋 Breakdown by Type:")
    for class_name, count in class_counts.items():
        print(f"   • {class_name}: {count}")
    print("\n" + "="*50)
    
    # Get confidence scores
    if total_count > 0:
        confidences = boxes.conf.cpu().numpy()
        avg_confidence = np.mean(confidences)
        min_confidence = np.min(confidences)
        max_confidence = np.max(confidences)
        
        print(f"\n📈 Confidence Statistics:")
        print(f"   • Average: {avg_confidence:.2%}")
        print(f"   • Range: {min_confidence:.2%} - {max_confidence:.2%}")

## 6. Visualize Results

Display the blueprint image with bounding boxes drawn around detected objects.

In [ ]:
# Plot results
for result in results:
    # Get annotated image
    annotated_img = result.plot(
        conf=True,        # Show confidence scores
        line_width=2,     # Bounding box line width
        font_size=12      # Label font size
    )
    
    # Convert BGR to RGB for matplotlib
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    # Display
    plt.figure(figsize=(16, 12))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f'Detection Results: {total_count} objects detected', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Optionally save the annotated image
    output_path = Path('../models/detection_result.jpg')
    cv2.imwrite(str(output_path), annotated_img)
    print(f"\n💾 Annotated image saved to: {output_path.absolute()}")

## 7. Detailed Detection Information

In [ ]:
# Display detailed information for each detection
print("\n📝 Detailed Detection List:")
print("="*70)

for result in results:
    boxes = result.boxes
    
    if len(boxes) == 0:
        print("No objects detected.")
    else:
        for idx, box in enumerate(boxes, 1):
            # Get box data
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            
            print(f"\n{idx}. {class_name.upper()}")
            print(f"   Confidence: {confidence:.2%}")
            print(f"   Location: ({int(x1)}, {int(y1)}) to ({int(x2)}, {int(y2)})")
            print(f"   Size: {int(x2-x1)} x {int(y2-y1)} pixels")

## 8. Batch Processing (Optional)

Process multiple images at once and generate a summary report.

In [ ]:
# Process all images in a directory
def process_directory(directory_path, output_csv=None):
    """
    Process all images in a directory and optionally save results to CSV.
    """
    import pandas as pd
    
    directory = Path(directory_path)
    image_files = list(directory.glob('*.png')) + list(directory.glob('*.jpg')) + list(directory.glob('*.jpeg'))
    
    if not image_files:
        print(f"No images found in {directory}")
        return
    
    print(f"\n📁 Processing {len(image_files)} images from {directory.name}...\n")
    
    results_data = []
    
    for img_path in image_files:
        # Run detection
        results = model(str(img_path), conf=0.25, verbose=False)
        
        for result in results:
            boxes = result.boxes
            total_count = len(boxes)
            
            # Count by class
            class_counts = {}
            for box in boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                class_counts[class_name] = class_counts.get(class_name, 0) + 1
            
            # Store results
            result_row = {
                'filename': img_path.name,
                'total_count': total_count,
                **class_counts
            }
            results_data.append(result_row)
            
            print(f"✅ {img_path.name}: {total_count} objects")
    
    # Create DataFrame
    df = pd.DataFrame(results_data).fillna(0)
    
    # Display summary
    print("\n" + "="*70)
    print("📊 BATCH PROCESSING SUMMARY")
    print("="*70)
    print(df.to_string(index=False))
    print("\n" + "="*70)
    print(f"Total objects across all images: {int(df['total_count'].sum())}")
    print("="*70)
    
    # Save to CSV if requested
    if output_csv:
        df.to_csv(output_csv, index=False)
        print(f"\n💾 Results saved to: {output_csv}")
    
    return df

# Example usage (uncomment to run):
# results_df = process_directory('../data_labeled/images', output_csv='../models/takeoff_results.csv')

## 9. Adjust Confidence Threshold (Optional)

If you're getting too many false positives or missing objects, adjust the confidence threshold.

In [ ]:
# Test different confidence thresholds
confidence_thresholds = [0.15, 0.25, 0.35, 0.50]

print("🔬 Testing different confidence thresholds:\n")

for conf_threshold in confidence_thresholds:
    results = model(test_image_path, conf=conf_threshold, verbose=False)
    
    for result in results:
        count = len(result.boxes)
        print(f"Confidence {conf_threshold:.0%}: {count} objects detected")

print("\n💡 Tip: Lower threshold = more detections (but more false positives)")
print("        Higher threshold = fewer detections (but more accurate)")

## Summary

✅ **MVP Deliverable Complete!**

You now have:
1. ✅ A trained YOLOv8 model for construction element detection
2. ✅ Inference capability on blueprint images
3. ✅ **Automated object counting for takeoff quantities**
4. ✅ Visual output with bounding boxes
5. ✅ Batch processing capability

### Next Steps:
- **Improve accuracy**: Add more labeled training data
- **Add more classes**: Detect doors, windows, fixtures, etc.
- **Optimize performance**: Try larger models (yolov8s, yolov8m) for better accuracy
- **Production deployment**: Create a web interface or API
- **Integration**: Connect to your takeoff/estimation software

### Production Considerations:
- Save detection results to a database
- Generate PDF reports with annotated images
- Track confidence scores for quality control
- Implement human review workflow for low-confidence detections